In [1]:
import numpy as np
import pandas as pd
import zipfile
import dask.dataframe as dk
import tensorflow as tf
import io
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
# temp_dir = "C:/Users/hoang/FileCSV_DACN_2025/ddos_dos_"

input_files = [f"file{i+1}.csv" for i in range(3)]
temp_dir =  "FL_Dataset/"

input_files = [temp_dir + output_file for output_file in input_files]
print(input_files)
df = [dk.read_csv(file) for file in input_files]
test_df = dk.read_csv("FL_Dataset/test.csv")
# input_zip = "/mnt/c/Users/hoang/FileCSV_DACN_2025/2Type.zip"
# csv_files = []
# with zipfile.ZipFile(input_zip, 'r') as z:
#     csv_files = [f for f in z.namelist() if f.endswith('.csv')]
# print(csv_files)
# df = [dk.read_csv(f'zip://{file}::{input_zip}') for file in csv_files]
# print(df[0].shape)

2025-05-07 09:59:17.009011: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-07 09:59:17.177439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746586757.261595     647 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746586757.293997     647 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746586757.426022     647 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

['FL_Dataset/file1.csv', 'FL_Dataset/file2.csv', 'FL_Dataset/file3.csv']


In [2]:
batch_size = 512
ratio_test_all = 0.15

from dask_ml.model_selection import train_test_split 
# Split 
train_df, val_df = train_test_split(df, test_size=ratio_test_all, random_state=42, shuffle=True)

# def dask_to_tf_dataset(dask_df, batch_size, num_classes): 
#     def generator():
#         for batch in dask_df.to_delayed():
#             batch=batch.compute()  
#             if batch.empty:
#                 continue

#             X = batch.drop(columns='label').values.astype(np.float32)
#             y = batch['label'].values
#             y_onehot = to_categorical(y, num_classes=num_classes)  

#             num_splits = max(1, len(X) // batch_size)  # Đảm bảo không chia nhỏ quá mức
#             X_batches = np.array_split(X, num_splits)
#             y_batches = np.array_split(y_onehot, num_splits)

#             for X_batch, y_batch in zip(X_batches, y_batches):
#                 yield X_batch, y_batch
                
#     output_signature = ( 
#         tf.TensorSpec(shape=(None, 46), dtype=tf.float32), 
#         tf.TensorSpec(shape=(None, 3), dtype=tf.int32),
#     )
    
#     return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)


############## binary ########################
# # load từng batch
def dask_to_tf_dataset(dask_df, batch_size): 
    def generator():
        for batch in dask_df.to_delayed():
            batch = batch.compute()  
            if batch.empty:
                continue

            X = batch.drop(columns='label').values.astype(np.float32)
            y = batch['label'].values.astype(np.int32)  # nhị phân: 0 hoặc 1

            num_splits = max(1, len(X) // batch_size)
            X_batches = np.array_split(X, num_splits)
            y_batches = np.array_split(y, num_splits)

            for X_batch, y_batch in zip(X_batches, y_batches):
                yield X_batch, y_batch

    output_signature = ( 
        tf.TensorSpec(shape=(None, 46), dtype=tf.float32), 
        tf.TensorSpec(shape=(None,), dtype=tf.int32),  # không phải one-hot nữa
    )
    
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)
############ binary ###########

In [3]:
# train_df1, test_df1 = df1.random_split([1 - ratio_test_all, ratio_test_all])
# train_df2, test_df2 = df2.random_split([1 - ratio_test_all, ratio_test_all])
# train_df3, test_df3 = df3.random_split([1 - ratio_test_all, ratio_test_all])
train_dfs = []
val_dfs = []
for dff in df:
    train_df, val_df =dff.random_split([1 - ratio_test_all, ratio_test_all])
    train_dfs.append(train_df)
    val_dfs.append(val_df)
   

train_gens = [dask_to_tf_dataset(train_df, 512).repeat() for train_df in train_dfs]
val_gens = [dask_to_tf_dataset(val_df , 512) for val_df in val_dfs]
test_gens = dask_to_tf_dataset(test_df , 512)

I0000 00:00:1746586763.272350     647 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [4]:
import datetime
import numpy as np
import tenseal as ts
#
from server import Server
from client import Client
num_servers = 1
num_clients = 3

stepsPerEpoch_Clients = [int( np.ceil(train_dfs[index].shape[0] / batch_size)) for index in range(num_clients)]
stepsValidate_Clients = [int( np.ceil(val_dfs[index].shape[0] / batch_size)) for index in range(num_clients)]
stepsTest_Clients = int( np.ceil(test_df.shape[0] / batch_size))
active_servers_list  = ['server_'+str(i)\
                        for i in range(num_servers)]
active_clients_list  = ['client_'+str(i)\
                        for i in range(num_clients)]

print(active_servers_list)
print(active_clients_list)

def init_he_context():
    """Thiết lập context mã hóa đồng hình"""
    context = ts.context(
        ts.SCHEME_TYPE.CKKS, # ckks cho số thực, bfv cho int
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
    context.generate_galois_keys()
    context.global_scale = 2**40
    return context

context = init_he_context()
agents_dict= {}
serverObjects={}
clientObjects={}
serverObjects = {server_name: Server(server_name=server_name, \
                        active_clients_list=active_clients_list) \
                        for server_name in active_servers_list}

clientObjects = {client_name: Client(client_name, train_gens[clientID], val_gens[clientID], test_gens, \
                        active_clients_list = active_clients_list, he_context=context) \
                        for clientID, client_name in enumerate(active_clients_list)}

for index, client_name in enumerate(active_clients_list):
    clientObjects[client_name].set_steps_per_epoch(stepsPerEpoch_Clients[index])
    clientObjects[client_name].get_steps_per_epoch()
    clientObjects[client_name].set_validation_steps(stepsValidate_Clients[index])
    clientObjects[client_name].get_validation_steps()
    clientObjects[client_name].set_test_steps(stepsTest_Clients)
    clientObjects[client_name].get_test_steps()
    
# lưu dict
agents_dict['server'] = serverObjects
agents_dict['client'] = clientObjects

# init agents_dict vừa tạo vào client, server
for agent_name, agent in serverObjects.items():
    agent.set_agentsDict(agents_dict=agents_dict)
for agent_name, agent in clientObjects.items():
    agent.set_agentsDict(agents_dict=agents_dict)

client_name = 'client_1'
print("Agent_Dict: ", agents_dict['client'][client_name])

server = agents_dict['server']['server_0']
print(server)

['server_0']
['client_0', 'client_1', 'client_2']
Train steps:  1693
Val steps:  299
Test steps:  1992
Train steps:  2539
Val steps:  449
Test steps:  1992
Train steps:  2540
Val steps:  448
Test steps:  1992
Agent_Dict:  <client.Client object at 0x7f058e943f40>


In [ ]:
if __name__ == '__main__':
    server.InitLoop()
    server.final_statistics()

====================================== Đang chạy Iteration 1 ======================================
Epoch 1/15
Epoch 1/15
Epoch 1/15


I0000 00:00:1746586776.112082     703 service.cc:152] XLA service 0x7f042d0bd1e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746586776.112121     703 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-05-07 09:59:36.171728: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746586776.493952     706 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-07 09:59:37.754778: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1332', 132 bytes spill stores, 132 bytes spill loads

2025-05-07 09:59:38.546901: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fus

   1/2540 ━━━━━━━━━━━━━━━━━━━━ 9:14:00 13s/step - accuracy: 0.2977 - loss: 1.3202

I0000 00:00:1746586784.618721     706 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 311/2540 ━━━━━━━━━━━━━━━━━━━━ 36s 16ms/step - accuracy: 0.9856 - loss: 0.0276

...

1693/1693 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.9982 - loss: 0.0206 - val_accuracy: 0.8915 - val_loss: 0.4837
Epoch 14/15
1890/2539 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9999 - loss: 0.0213

2025-05-07 10:13:22.501552: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 15685740104855525371
2025-05-07 10:13:22.501878: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 17549315412094336176
/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2540/2540 ━━━━━━━━━━━━━━━━━━━━ 36s 14ms/step - accuracy: 0.9990 - loss: 0.0151 - val_accuracy: 0.8929 - val_loss: 1.6318
Epoch 10/15
  12/2540 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 1.0000 - loss: 3.8672e-04 

2025-05-07 10:13:31.487903: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 15685740104855525371
2025-05-07 10:13:31.487955: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 17549315412094336176
/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1545/1693 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9986 - loss: 0.0188

In [ ]:
from tensorflow.keras.models import load_model
# tempdirs = ["D:/DoAnChuyenNganh_Train/client_0_log/11h18p__02-04-2025/", "D:/DoAnChuyenNganh_Train/client_1_log/11h18p__02-04-2025/",  "D:/DoAnChuyenNganh_Train/client_2_log/11h18p__02-04-2025/"]
timeFolder="14h24p__15-04/"
tempdirs = [f"D:/DoAnChuyenNganh_Train/log/client_{i}_log/" for i in range(len(active_clients_list))]


model_names =[timeFolder+f"model_{i+1}.keras" for i in range(5)]
print(model_names)
models = {}

for i, client_name in enumerate(active_clients_list):
    models[client_name] = [load_model(tempdirs[i]+model_name) for model_name in model_names]
print (models['client_0'])

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.models import load_model

import numpy as np

num_batch_test_eachClient  =  []
for index, test_df in enumerate(test_dfs):
    num_samples_test = test_df.shape[0].compute()
    # Tính số batch
    num_batches_test = int(np.ceil(num_samples_test / batch_size))
    num_batch_test_eachClient.append(num_batches_test)

print("Num Batch Each Client: ", num_batch_test_eachClient)
X_tests = {}
Y_tests = {}
Y_preds= {}
for i, client_name in enumerate(active_clients_list):
    Y_tests[client_name]={}
    Y_preds[client_name]={}

# for i, client_name in enumerate(active_clients_list):
#     X_test = []
#     y_test = []
#     for X_batch, y_batch in test_gens[i].take(num_batch_test_eachClient[i]):
#         X_test.extend(X_batch.numpy().flatten())
#         y_test.extend(y_batch.numpy().flatten())
#         y_pred = []
#         for iteration in range(5):    
#     # .as_numpy_iterator():
#         # # take(12000):
#         # X_test_list.append(X_batch.numpy())
#         # y_test_list.append(y_batch.numpy())  # .numpy()

#         # # Gộp tất cả batch lại
#         # X_test = np.concatenate(X_test_list, axis=0)
#         # y_test = np.concatenate(y_test_list, axis=0)

#         # # Nếu y_test đang ở dạng one-hot, chuyển về dạng số
#         # y_test = np.argmax(y_test, axis=1)
#             y_pred_pre = models[client_name][iteration].predict(X_batch, verbose=0)
#             y_pred.extend((y_pred_pre > 0.5).astype(int).flatten())
    
#         Y_tests[client_name][iteration] = y_test
#         Y_preds[client_name][iteration] = y_pred
    
# print(Y_preds)

In [ ]:
# Client 0
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

precisions = []
recalls = []
f1s = []

Iterations = [f"Iteration {index+1}" for index in range(5)]

X_test = []
y_test = []
for X_batch, y_batch in test_gens[i].take(num_batch_test_eachClient[0]):
    X_test.append(X_batch.numpy())
    y_test.append(y_batch.numpy())
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

print(len(X_test))
for iteration in range(len(Iterations)):
    print(models[client_name][iteration])
    y_pred_pre = models[client_name][iteration].predict(X_test, verbose=1)
    y_pred = (y_pred_pre > 0.5).astype(int).flatten()
    
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    recalls.append(recall_score(y_test, y_pred, average='binary'))
    f1s.append(f1_score(y_test, y_pred, average='binary'))


In [ ]:
print("Precision Score: ", precisions)
print("Recall Score: ", recalls)
print("F1 Score: ", f1s)  


# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 0")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()


attack_types = ['BenignTraffic', 'DoS&DDoS']
cm = confusion_matrix(y_test, y_pred)
    # Vẽ heatmap
plt.figure(figsize=(12, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=attack_types, yticklabels=attack_types)
plt.yticks(rotation=360)
# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 0")

# Hiển thị
plt.show()


# attack_types = ['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
#                    'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']

metrics = []

# Số lượng lớp (10 lớp)
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(12, 5), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 0")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client 1
import matplotlib.pyplot as plt

precisions = []
recalls = []
f1s = []
y_pred = []

Iterations = [f"Iteration {index+1}" for index in range(5)]

X_test = []
y_test = []

print(num_batch_test_eachClient[1])

for X_batch, y_batch in test_gens[1].take(num_batch_test_eachClient[1]):
    X_test.append(X_batch.numpy())
    y_test.append(y_batch.numpy())
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

for iteration in range(len(Iterations)):
    y_pred_pre = models['client_1'][iteration].predict(X_test, verbose=1)
    y_pred = (y_pred_pre > 0.5).astype(int).flatten()
    
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    recalls.append(recall_score(y_test, y_pred, average='binary'))
    f1s.append(f1_score(y_test, y_pred, average='binary'))
    
print("Precision Score: ", precisions)
print("Recall Score: ", recalls)
print("F1 Score: ", f1s)  


# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 1")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()


attack_types = ['BenignTraffic', 'DoS&DDoS']
cm = confusion_matrix(y_test, y_pred)
    # Vẽ heatmap
plt.figure(figsize=(12, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=attack_types, yticklabels=attack_types)
plt.yticks(rotation=360)
# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 1")

# Hiển thị
plt.show()

# attack_types = ['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
#                    'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']

metrics = []

# Số lượng lớp (10 lớp)
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(12, 5), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 1")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client 2
import matplotlib.pyplot as plt

precisions = []
recalls = []
f1s = []

y_pred = []
Iterations = [f"Iteration {index+1}" for index in range(5)]

X_test = []
y_test = []
for X_batch, y_batch in test_gens[2].take(num_batch_test_eachClient[2]):
    X_test.append(X_batch.numpy())
    y_test.append(y_batch.numpy())
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

for iteration in range(len(Iterations)):
    y_pred_pre = models['client_2'][iteration].predict(X_test, verbose=1)
    y_pred = (y_pred_pre > 0.5).astype(int).flatten()
    
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    recalls.append(recall_score(y_test, y_pred, average='binary'))
    f1s.append(f1_score(y_test, y_pred, average='binary'))
    
print("Precision Score: ", precisions)
print("Recall Score: ", recalls)
print("F1 Score: ", f1s)  


# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 2")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()


attack_types = ['BenignTraffic', 'DoS&DDoS']
cm = confusion_matrix(y_test, y_pred)
    # Vẽ heatmap
plt.figure(figsize=(12, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=attack_types, yticklabels=attack_types)
plt.yticks(rotation=360)
# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 2")

# Hiển thị
plt.show()


# attack_types = ['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
#                    'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']

metrics = []

# Số lượng lớp (10 lớp)
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(12, 5), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 2")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client_0 
import matplotlib.pyplot as plt
import numpy as np

file_dir= "log/client_0_log/14h24p__15-04/"
file_names = [f"Iteration_{index+1}.csv" for index in range(5)]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(18, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

In [ ]:
# Client_1
import matplotlib.pyplot as plt
import numpy as np

file_dir= "log/client_1_log/14h24p__15-04/"
file_names = [f"Iteration_{index+1}.csv" for index in range(5)]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(18, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

In [ ]:
# Client_2
import matplotlib.pyplot as plt
import numpy as np

file_dir= "log/client_2_log/14h24p__15-04/"
file_names = [f"Iteration_{index+1}.csv" for index in range(5)]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(18, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs _Client 2")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs Client 2")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()